In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
tqdm.tqdm.pandas()

In [2]:
date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')

In [3]:
date_today_record = str(date_today).split(' ')[0].replace('-', '')
date_today_record

'20220222'

In [4]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [5]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [6]:
def data_prep(bdtype, tradetype, starting_year=2018):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    if (tradetype != '매매') & (tradetype != '전월세'):
        raise ValueError('두번째 변수는 매매 또는 전월세만 입력 가능')
    
    
    basedir = './국토교통부_실거래가_공개시스템/경기도/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    
    dfs_list = []
    for i, f in tqdm.tqdm_notebook(enumerate(filenames)):
        splitted_filename = f.split('실거래가_')       
        file_year = int(splitted_filename[-1][:4])
        
        '''if file_year < starting_year:
            continue'''
        
        try:
            df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        except:
            try:
                df = pd.read_csv(basedir + f, encoding='utf-8', header=15)
            except:
                df = pd.read_csv(basedir + f, header=15)
        
        if '건물명' in df.columns:
            df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
        elif '단지명' in df.columns:
            df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
            
        if '대지권면적(㎡)' in df.columns:
            df = df.drop(columns=['대지권면적(㎡)'])
                
            
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])
            
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    if tradetype == '전월세':
        concat_df = concat_df.rename(columns={'전월세구분':'거래구분', '보증금(만원)':'거래금액(만원)'})
        concat_df = concat_df.drop(columns=['월세(만원)'])
        concat_df = concat_df[concat_df['거래구분'] == '전세']
    elif tradetype == '매매':
        concat_df['거래구분'] = '매매'
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].astype('Int64')
    concat_df['계약일'] = concat_df['계약일'].astype('Int64')
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['번지', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [7]:
apart_df = data_prep('아파트', '매매')
apart_df['부동산유형'] = '아파트'
print(apart_df.shape)
apart_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


(692425, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,경기도 가평군 가평읍 달전리 281-0,달전로 59,경기도 가평군 가평읍 달전리,281,0,세대넥스빌,59.88,10500,10,1996.0,-,-,매매,22.0,2018-01-16,175.350701,26.0,아파트
1,경기도 가평군 가평읍 달전리 281-0,달전로 59,경기도 가평군 가평읍 달전리,281,0,세대넥스빌,59.88,10500,10,1996.0,-,-,매매,22.0,2018-01-18,175.350701,26.0,아파트
2,경기도 가평군 가평읍 달전리 281-0,달전로 59,경기도 가평군 가평읍 달전리,281,0,세대넥스빌,58.95,10700,15,1996.0,-,-,매매,22.0,2018-02-13,181.509754,26.0,아파트
3,경기도 가평군 가평읍 달전리 281-0,달전로 59,경기도 가평군 가평읍 달전리,281,0,세대넥스빌,71.43,9400,3,1996.0,-,-,매매,22.0,2018-02-22,131.597368,26.0,아파트
4,경기도 가평군 가평읍 달전리 281-0,달전로 59,경기도 가평군 가평읍 달전리,281,0,세대넥스빌,84.84,16300,4,1996.0,-,-,매매,22.0,2018-02-28,192.126355,26.0,아파트


In [8]:
yunrip_df = data_prep('연립다세대', '매매')
yunrip_df['부동산유형'] = '연립다세대'
print(yunrip_df.shape)
yunrip_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(196846, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,60.58,6200,1,2007.0,-,-,매매,11.0,2018-05-16,102.344008,15.0,연립다세대
1,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,76.92,9400,2,2007.0,-,-,매매,11.0,2018-11-15,122.204888,15.0,연립다세대
2,경기도 가평군 가평읍 달전리 422-1,달전로 8-5,경기도 가평군 가평읍 달전리,422,1,금오빌라가동,59.94,7500,1,1991.0,-,-,매매,27.0,2018-07-03,125.125125,31.0,연립다세대
3,경기도 가평군 가평읍 달전리 272-11,달전로 83,경기도 가평군 가평읍 달전리,272,11,남이주택나동,48.80,4500,1,1992.0,-,-,매매,26.0,2018-02-14,92.213115,30.0,연립다세대
4,경기도 가평군 가평읍 달전리 272-2,달전로 83,경기도 가평군 가평읍 달전리,272,2,남이주택다동,35.46,4900,1,1992.0,-,-,매매,26.0,2018-03-09,138.183869,30.0,연립다세대


In [9]:
officetel_df = data_prep('오피스텔', '매매')
officetel_df['부동산유형'] = '오피스텔'
print(officetel_df.shape)
officetel_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(50842, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,경기도 가평군 청평면 청평리 301-4,경춘로 871,경기도 가평군 청평면 청평리,301,4,신공간사무주택,41.31,3700,5,1996.0,-,-,매매,22.0,2018-07-31,89.566691,26.0,오피스텔
1,경기도 고양덕양구 동산동 369-0,동송로 33,경기도 고양덕양구 동산동,369,0,이편한세상시티삼송,77.51,41122,24,2018.0,-,-,매매,0.0,2018-05-29,530.537995,4.0,오피스텔
2,경기도 고양덕양구 동산동 369-0,동송로 33,경기도 고양덕양구 동산동,369,0,이편한세상시티삼송,54.32,28734,12,2018.0,-,-,매매,0.0,2018-06-30,528.976436,4.0,오피스텔
3,경기도 고양덕양구 삼송동 289-0,삼송로 210,경기도 고양덕양구 삼송동,289,0,현대썬앤빌,24.65,14829,9,2018.0,-,-,매매,0.0,2018-11-14,601.582150,4.0,오피스텔
4,경기도 고양덕양구 성사동 696-0,호국로 798,경기도 고양덕양구 성사동,696,0,삼성홈타워오피스텔,32.90,9300,6,2006.0,-,-,매매,12.0,2018-01-05,282.674772,16.0,오피스텔


In [10]:
house_df = pd.concat([apart_df, yunrip_df, officetel_df]).sort_values(['시군구', '본번', '부번', '건축년도', '전용면적(㎡)', '계약날짜']).reset_index(drop=True)
house_df = house_df.drop(columns=['거래유형', '중개사소재지', '거래구분', '계약날짜기준_건물연식'])
del apart_df, yunrip_df, officetel_df
print(house_df.shape)
house_df.head()

(940113, 14)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,60.58,6200,1,2007.0,2018-05-16,102.344008,15.0,연립다세대
1,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,60.58,6800,3,2007.0,2019-05-14,112.248267,15.0,연립다세대
2,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,76.92,9400,2,2007.0,2018-11-15,122.204888,15.0,연립다세대
3,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,76.92,8500,3,2007.0,2020-10-23,110.504420,15.0,연립다세대
4,경기도 가평군 가평읍 경반리 240-1,경반안로116번길 22-53,경기도 가평군 가평읍 경반리,240,1,삼성세르빌,76.92,9900,2,2007.0,2020-12-16,128.705148,15.0,연립다세대


In [11]:
'''land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()'''

"land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')\nprint(land_specs_df.shape)\nland_specs_df.head()"

In [12]:
'''last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)
print(last_df.shape)
last_df.head()'''

"last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)\nprint(last_df.shape)\nlast_df.head()"

In [13]:
'''house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)
print(house_merge_df.shape)'''

"house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)\nprint(house_merge_df.shape)"

In [14]:
house_df['일괄계약'] = 'N'

In [15]:
#house_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)

In [16]:
'''print(house_df.shape)
house_df.head()'''

'print(house_df.shape)\nhouse_df.head()'

In [17]:
def identify_bulk_contract(df):
    df = df.copy()
    
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        addr_df = df[df['지번주소'] == addr]
        
        for btyear in addr_df['건축년도'].unique():
            btyear_df = addr_df[addr_df['건축년도'] == btyear]
            
            if (btyear_df.shape[0] >= 10) & (btyear_df['계약날짜'].nunique() == 1):
                df.loc[btyear_df.index, '일괄계약'] = 'Y'

    return df

In [18]:
house_df['계약날짜'].describe()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                  940113
unique                   1513
top       2020-06-16 00:00:00
freq                     2998
first     2018-01-01 00:00:00
last      2022-02-21 00:00:00
Name: 계약날짜, dtype: object

In [19]:
house_df = identify_bulk_contract(house_df)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/62991 [00:00<?, ?it/s]

In [20]:
bulk_df = house_df[house_df['일괄계약'] == 'Y']
print(bulk_df.shape)
bulk_df.head()

(19561, 15)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약
361,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",50.070,8200,5,2013.0,2020-03-18,163.770721,9.0,연립다세대,Y
362,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",57.195,8400,5,2013.0,2020-03-18,146.865985,9.0,연립다세대,Y
363,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,2,2013.0,2020-03-18,146.520147,9.0,연립다세대,Y
364,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,3,2013.0,2020-03-18,146.520147,9.0,연립다세대,Y
365,경기도 가평군 가평읍 읍내리 266-1,가화로 154-1,경기도 가평군 가평읍 읍내리,266,1,"101동,102동",58.695,8600,4,2013.0,2020-03-18,146.520147,9.0,연립다세대,Y


In [21]:
bulk_df['지번주소'].nunique()

986

In [22]:
%%time
bulk_df.to_excel('./공공주택찾기/경기도/일괄계약전체_20180101_20220222.xlsx', index=False)

Wall time: 5.43 s
